In [26]:

from keras import layers
from keras import models
from keras.datasets import mnist
from keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
import cv2
import numpy as np

In [27]:
# Каталог с данными для обучения
train_dir = 'cadmnist/train'
# Каталог с данными для проверки
val_dir = 'cadmnist/val'
# Каталог с данными для тестирования
test_dir = 'cadmnist/test'
# Размеры изображения
img_width, img_height = 150, 150
# Размерность тензора на основе изображения для входных данных в нейронную сеть
# backend Tensorflow, channels_last
input_shape = (img_width, img_height, 1)
# Количество эпох
epochs = 10
# Размер мини-выборки
batch_size = 8
# Количество изображений для обучения
nb_train_samples = 277
# Количество изображений для проверки
nb_validation_samples = 86
# Количество изображений для тестирования
# nb_test_samples = 86
# количество классов
num_classes=5

In [28]:
model = models.Sequential()
# model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
# model.add(layers.MaxPooling2D((2, 2)))
# model.add(layers.Conv2D(64, (3, 3), activation='relu'))
# model.add(layers.MaxPooling2D((2, 2)))
# model.add(layers.Conv2D(64, (3, 3), activation='relu'))

# model.add(layers.Flatten())
# model.add(layers.Dense(64, activation='relu'))
# model.add(layers.Dense(num_classes, activation='softmax'))

model.add(layers.Conv2D(32, (3, 3), input_shape=input_shape))
model.add(layers.Activation('relu'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))

model.add(layers.Conv2D(32, (3, 3)))
model.add(layers.Activation('relu'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))

model.add(layers.Conv2D(64, (3, 3)))
model.add(layers.Activation('relu'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))

model.add(layers.Flatten())
model.add(layers.Dense(64))
model.add(layers.Activation('relu'))
model.add(layers.Dense(5))
model.add(layers.Activation('softmax'))
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_9 (Conv2D)           (None, 148, 148, 32)      320       
                                                                 
 activation_15 (Activation)  (None, 148, 148, 32)      0         
                                                                 
 max_pooling2d_9 (MaxPooling  (None, 74, 74, 32)       0         
 2D)                                                             
                                                                 
 conv2d_10 (Conv2D)          (None, 72, 72, 32)        9248      
                                                                 
 activation_16 (Activation)  (None, 72, 72, 32)        0         
                                                                 
 max_pooling2d_10 (MaxPoolin  (None, 36, 36, 32)       0         
 g2D)                                                 

In [29]:
datagen = ImageDataGenerator()

train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(img_width, img_height),
    color_mode='grayscale',
    batch_size=batch_size,
    class_mode='categorical')

val_generator = datagen.flow_from_directory(
    val_dir,
    target_size=(img_width, img_height),
    color_mode='grayscale',
    batch_size=batch_size,
    class_mode='categorical')

# test_generator = datagen.flow_from_directory(
#     test_dir,
#     target_size=(img_width, img_height),
#     color_mode='grayscale',
#     batch_size=batch_size,
#     class_mode='categorical')

Found 277 images belonging to 5 classes.
Found 86 images belonging to 5 classes.


In [30]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [31]:
model.fit(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=val_generator,
    validation_steps=nb_validation_samples // batch_size)

Epoch 1/10
34/34 [==============================] - 5s 141ms/step - loss: 18.4519 - accuracy: 0.4796 - val_loss: 1.1000 - val_accuracy: 0.6250
Epoch 2/10
34/34 [==============================] - 5s 139ms/step - loss: 0.6610 - accuracy: 0.8141 - val_loss: 0.8454 - val_accuracy: 0.7375
Epoch 3/10
34/34 [==============================] - 5s 137ms/step - loss: 0.3394 - accuracy: 0.9071 - val_loss: 0.3268 - val_accuracy: 0.9125
Epoch 4/10
34/34 [==============================] - 5s 134ms/step - loss: 0.1753 - accuracy: 0.9554 - val_loss: 0.3513 - val_accuracy: 0.9250
Epoch 5/10
34/34 [==============================] - 5s 134ms/step - loss: 0.1177 - accuracy: 0.9777 - val_loss: 0.3498 - val_accuracy: 0.9250
Epoch 6/10
34/34 [==============================] - 5s 133ms/step - loss: 0.0305 - accuracy: 0.9963 - val_loss: 0.3405 - val_accuracy: 0.9500
Epoch 7/10
34/34 [==============================] - 5s 136ms/step - loss: 0.0104 - accuracy: 1.0000 - val_loss: 0.3668 - val_accuracy: 0.9375
Epoch

In [32]:
# scores = model.evaluate_generator(test_generator, nb_test_samples // batch_size)
# print("Точность на тестовых данных: %.2f%%" % (scores[1]*100))

In [33]:
model.save('cadmnist.h5')

In [35]:
model = models.load_model('cadmnist.h5')
classes=["bearing", "bolt", "nut", "seal", "washer"]

img=cv2.imread('cadmnist/val/seals/502164.jpg', cv2.IMREAD_GRAYSCALE).reshape((1,150,150,1))
img=img/255.0
pred = model.predict(img)[0]
final_pred = np.argmax(pred)
data = classes[final_pred] + ' ' + str(int(max(pred) * 100)) + '%'
print(pred)
print(data)

img=cv2.imread('cadmnist/val/bolts/022162.jpg', cv2.IMREAD_GRAYSCALE).reshape((1,150,150,1))
img=img/255.0
pred = model.predict(img)[0]
final_pred = np.argmax(pred)
data = classes[final_pred] + ' ' + str(int(max(pred) * 100)) + '%'
print(pred)
print(data)

img=cv2.imread('cadmnist/val/nuts/312161.jpg', cv2.IMREAD_GRAYSCALE).reshape((1,150,150,1))
img=img/255.0
pred = model.predict(img)[0]
final_pred = np.argmax(pred)
data = classes[final_pred] + ' ' + str(int(max(pred) * 100)) + '%'
print(pred)
print(data)

img=cv2.imread('cadmnist/val/bearings/129164.jpg', cv2.IMREAD_GRAYSCALE).reshape((1,150,150,1))
img=img/255.0
pred = model.predict(img)[0]
final_pred = np.argmax(pred)
data = classes[final_pred] + ' ' + str(int(max(pred) * 100)) + '%'
print(pred)
print(data)

img=cv2.imread('cadmnist/val/washers/391164.jpg', cv2.IMREAD_GRAYSCALE).reshape((1,150,150,1))
img=img/255.0
pred = model.predict(img)[0]
final_pred = np.argmax(pred)
data = classes[final_pred] + ' ' + str(int(max(pred) * 100)) + '%'
print(pred)
print(data)

1/1 [==============================] - 0s 101ms/step
[0.19788478 0.20047295 0.19655852 0.20709549 0.19798826]
seal 20%
1/1 [==============================] - 0s 30ms/step
[0.19763426 0.2069869  0.19719604 0.20270795 0.19547483]
bolt 20%
1/1 [==============================] - 0s 29ms/step
[0.20172702 0.20061304 0.20163016 0.20116086 0.19486892]
bearing 20%
1/1 [==============================] - 0s 28ms/step
[0.20688897 0.19448136 0.19582534 0.20636976 0.19643462]
bearing 20%
1/1 [==============================] - 0s 31ms/step
[0.19652756 0.20167756 0.19442329 0.20373552 0.20363605]
seal 20%
